In [27]:
# all imports here
import csv
from operator import itemgetter
import pandas as pd
import numpy as np
from collections import defaultdict
import humanize as hm
# imports and setting for plotly
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [28]:
def write_graph(graph_file, data, graph_name):
    graph_file.write("<!-- START of graph: " + graph_name + "-->\n\n")
    graph_file.write(data)
    graph_file.write("\n\n")
    graph_file.write("<br><br>\n\n")
    graph_file.write("<hr>\n\n")
    graph_file.write("<br><br>\n\n")
    graph_file.write("<!-- END of graph -->\n\n")



target_file = open("graphs_for_v6.0.3.html", 'w')
target_file.write('<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>\n\n')
target_file.write('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">\n\n')
target_file.write('<div class="container"> \n \t <div class="row row-centered"> \n\n')

In [29]:
def read_from_file(file, actors_amount):

    if actors_amount == 6:

        movies = {}

        with open(file) as csvfile:
            reader = csv.DictReader(csvfile, delimiter="\t")
            for entry in reader:
                movies[
                    entry["title"]
                ] = {
                    "director": entry["director"],
                    "rating": entry["rating"],
                    "votes": entry["votes"],
                    "year": entry["year"],
                    "genre": entry["genre"],
                    "gross": entry["gross"],
                    "budget": entry["budget"],
                    "run-time": entry["run-time"],
                    "actor1": entry["actor1"],
                    "actor1_rank": entry["actor1_rank"],
                    "actor1_sex": entry["actor1_sex"],
                    "actor2": entry["actor2"],
                    "actor2_rank": entry["actor2_rank"],
                    "actor2_sex": entry["actor2_sex"],
                    "actor3": entry["actor3"],
                    "actor3_rank": entry["actor3_rank"],
                    "actor3_sex": entry["actor3_sex"],
                    "actor4": entry["actor4"],
                    "actor4_rank": entry["actor4_rank"],
                    "actor4_sex": entry["actor4_sex"],
                    "actor5": entry["actor5"],
                    "actor5_rank": entry["actor5_rank"],
                    "actor5_sex": entry["actor5_sex"],
                    "actor6": entry["actor6"],
                    "actor6_rank": entry["actor6_rank"],
                    "actor6_sex": entry["actor6_sex"],
                    "plot": entry["plot"]
                }
        return movies

    if actors_amount == 3:

        movies = {}

        with open(file) as csvfile:
            reader = csv.DictReader(csvfile, delimiter="\t")
            for entry in reader:
                movies[
                    entry["title"]
                ] = {
                    "director": entry["director"],
                    "rating": entry["rating"],
                    "votes": entry["votes"],
                    "year": entry["year"],
                    "genre": entry["genre"],
                    "gross": entry["gross"],
                    "budget": entry["budget"],
                    "run-time": entry["run-time"],
                    "actor1": entry["actor1"],
                    "actor1_rank": entry["actor1_rank"],
                    "actor1_sex": entry["actor1_sex"],
                    "actor2": entry["actor2"],
                    "actor2_rank": entry["actor2_rank"],
                    "actor2_sex": entry["actor2_sex"],
                    "actor3": entry["actor3"],
                    "actor3_rank": entry["actor3_rank"],
                    "actor3_sex": entry["actor3_sex"],
                    "plot": entry["plot"]
                }
        return movies
    
# movies_main = read_from_file('imdb_dataset_v6.0.2_3_actors_comple.tsv', 3)
movies_main = read_from_file('imdb_dataset_v6.0.3_test_3_actors_complete.tsv', 3)


In [30]:
def get_dataset_info(dataset):
    amount_movies = defaultdict(int)
    amount_movies_years = defaultdict(int)
    amount_actors = defaultdict(int)
    amount_sex = defaultdict(int)
    
    for title in dataset:
        movie_info = dataset[title]
        
        amount_movies[title] += 1
        amount_movies_years[movie_info["year"]] += 1
        
        amount_actors[movie_info["actor1"]] += 1
        amount_actors[movie_info["actor2"]] += 1
        amount_actors[movie_info["actor3"]] += 1
        
        amount_sex[movie_info["actor1"]] = movie_info["actor1_sex"]
        amount_sex[movie_info["actor2"]] = movie_info["actor2_sex"]  
        amount_sex[movie_info["actor3"]] = movie_info["actor3_sex"]
        
#     print "Amount of all movies:",len(amount_movies)
    
    movies_years = ""
    movies_years_for_graph = []
    
    for entry in sorted(list(amount_movies_years.items()), key=itemgetter(0), reverse=False):
        movies_years = movies_years + entry[0] + " - " + str(entry[1]) + "<br>\n"
        movies_years_for_graph.append((entry[0], entry[1]))
        
#     print "Amount of different actors:", len(amount_actors)
    f, m = 0, 0
    for entry in amount_sex:
        if amount_sex[entry] == 'M':
            m+=1
        else:
            f+=1
    
#     print "Amount of actors:", m
#     print "Amount of actresses:", f
    
    return  ["<h1>Dataset info</h1> Amount of all movies: " + str(len(amount_movies)) + "<br>" + \
#             "Amount of movies in each year:<br>" + movies_years + "<br>" + \
            "Amount of all actors: " + str(len(amount_actors)),
            movies_years_for_graph ]
    
write_graph(target_file, get_dataset_info(movies_main)[0] , "dataset info")

In [31]:
results = get_dataset_info(movies_main)[1]

trace0 =go.Bar(
            x=[year for year, amount in results],
            y=[amount for year, amount in results]
    )


layout = go.Layout(
    title = "Amount of found movies in years 1986 - 2016",
    xaxis=dict(tickangle=-45,
               title = "Year"),
    yaxis=dict(title = "Amount"),
    showlegend=False,
    barmode='group',
    height=700,
    margin=go.Margin(
        l=200,
        r=50,
        b=250,
        t=100,
        pad=4 )
)

data = [trace0]

fig = go.Figure(data = data, layout=layout, )
iplot(fig)
write_graph(target_file, 
            plotly.offline.plot(fig, include_plotlyjs=False, output_type='div'),
           "movie years bar chart")

In [32]:
def count_genres(movies_data):
    genres = defaultdict(int)
    for title in movies_data:
        genres[movies_data[title]["genre"]] += 1
    return sorted(list(genres.items()), key=itemgetter(1), reverse=True)

results = count_genres(movies_main)

trace0 = go.Pie(
    labels = [genre for genre, count in results],
    values = [count for genre, count in results],
    hole = .2,
)

layout = go.Layout(
    title = "Movie Genres in dataset",
    height=600
)

data = [trace0]
fig = go.Figure(data = data, layout=layout, )
iplot(fig)

write_graph(target_file, 
            plotly.offline.plot(fig, include_plotlyjs=False, output_type='div'),
           "genres pie chart")

In [33]:
def find_budget_and_gross(movies_data):
    data = []
    for title in movies_data:
        data.append([title,
                     float(movies_data[title]["budget"]),
                     float(movies_data[title]["gross"])])
    result = sorted(data, key=itemgetter(2), reverse=True)
    return result

In [34]:
amount = 20
results = find_budget_and_gross(movies_main)[:amount]

trace0 =go.Bar(
            x=[title for title, budget, gross in results],
            y=[gross for title, budget, gross in results],
            name = "Gross"
    )

trace1 =go.Bar(
            x=[title for title, budget, gross in results],
            y=[budget for title, budget, gross in results],
            name = "Budget"
    )

layout = go.Layout(
    title = "Highest grossing movies (top 20)",
    xaxis=dict(tickangle=-45,
               title = "Movie title"),
    yaxis=dict(title = "USD"),
    showlegend=True,
    barmode='group',
    height=700,
    margin=go.Margin(
        l=200,
        r=50,
        b=250,
        t=100,
        pad=4 )
)

data = [trace0, trace1]

fig = go.Figure(data = data, layout=layout, )
iplot(fig)

write_graph(target_file, 
            plotly.offline.plot(fig, include_plotlyjs=False, output_type='div'),
           "highest gross")

In [35]:
def find_budget_and_gross_sort_by_budget(movies_data):
    data = []
    for title in movies_data:
        data.append([title,
                     float(movies_data[title]["budget"]),
                     float(movies_data[title]["gross"])])
    result = sorted(data, key=itemgetter(1), reverse=True)
    return result

amount = 20
results = find_budget_and_gross_sort_by_budget(movies_main)[:amount]

trace0 =go.Bar(
            x=[title for title, budget, gross in results],
            y=[gross for title, budget, gross in results],
            name = "Gross"
    )

trace1 =go.Bar(
            x=[title for title, budget, gross in results],
            y=[budget for title, budget, gross in results],
            name = "Budget"
    )

layout = go.Layout(
    title = "Movies with highest budget",
    xaxis=dict(tickangle=-45,
               title = "Movie title"),
    yaxis=dict(title = "USD"),
    showlegend=True,
    barmode='group',
    height=700,
    margin=go.Margin(
        l=200,
        r=50,
        b=250,
        t=100,
        pad=4 )
)

data = [trace0, trace1]

fig = go.Figure(data = data, layout=layout, )
iplot(fig)

write_graph(target_file, 
            plotly.offline.plot(fig, include_plotlyjs=False, output_type='div'),
           "highest budget")


In [36]:
results = find_budget_and_gross(movies_main)

# x = [budget for title, budget, gross in results]
# y = [gross for title, budget, gross in results]

# z = np.polyfit(x, y, 10)
# f = np.poly1d(z)

# x_new = np.linspace(x[0], x[-1], 100)
# # x_new = np.linspace(min(x),max(x) , 100)

# y_new = f(x_new)

trace0 = go.Scattergl(
    x = [budget for title, budget, gross in results],
    y = [gross for title, budget, gross in results],
    text = ["(Budget, Gross) for <br>" + title for title, budget, gross in results],
    mode='markers',
    marker=go.Marker(size=3.5,sizemode='area')
)

# trace1 = go.Scatter(
#                   x=x_new,
#                   y=y_new,
#                   mode='lines',
#                   marker=go.Marker(color='rgb(0, 0, 0)'),
#                   name='Fit'
#                   )

layout = go.Layout(
    title = "Budget vs Gross",
    hovermode="closest",
    xaxis=dict(tickangle=-45,
               title = "Budget USD"),
    yaxis=dict(title = "Gross USD"),
    showlegend=False,
    height=700,
)

# data = [ trace0, trace1]
data = [trace0]

fig = go.Figure(data = data, layout=layout, )
iplot(fig)

write_graph(target_file, 
            plotly.offline.plot(fig, include_plotlyjs=False, output_type='div'),
           "gross vs budget")


In [37]:
def actors_and_gross_for_all_their_movies_mean(movies_data):
    all_actors_gross = defaultdict(lambda :[])
    all_actors_budget = defaultdict(lambda: [])
    all_actors_amount = defaultdict(int)

    for title in movies_data:
        movie_info = movies_data[title]
        gross = float(movie_info["gross"])
        budget = float(movie_info["budget"])

        actor = movie_info["actor1"]
#         all_actors_gross[actor] += gross
#         all_actors_budget[actor] += budget
        all_actors_gross[actor].append(gross)
        all_actors_budget[actor].append(budget)
        all_actors_amount[actor] += 1

        actor = movie_info["actor2"]
#         all_actors_gross[actor] += gross
#         all_actors_budget[actor] += budget
        all_actors_gross[actor].append(gross)
        all_actors_budget[actor].append(budget)
        all_actors_amount[actor] += 1
        
        actor = movie_info["actor3"]
#         all_actors_gross[actor] += gross
#         all_actors_budget[actor] += budget
        all_actors_gross[actor].append(gross)
        all_actors_budget[actor].append(budget)
        all_actors_amount[actor] += 1
        
    all_actors_gross_mean = defaultdict(float)
    all_actors_budget_mean = defaultdict(float)
    
    for actor in all_actors_gross:
        all_actors_gross_mean[actor] = np.mean(all_actors_gross[actor])
        all_actors_budget_mean[actor] = np.mean(all_actors_budget[actor])

    
    return [
            all_actors_gross_mean,
            all_actors_budget_mean,
            all_actors_amount ]

result = actors_and_gross_for_all_their_movies_mean(movies_main)

res_gross_dict = result[0]
res_budge_dict = result[1]
res_amount_dict = result[2]

res_gross = [(key, res_gross_dict[key]) for key in sorted(res_gross_dict.iterkeys())]
res_budge = [(key, res_budge_dict[key]) for key in sorted(res_budge_dict.iterkeys())]
res_amount = [(key, res_amount_dict[key]) for key in sorted(res_amount_dict.iterkeys())]

actors_data = []

for _actor in res_gross_dict:
    actors_data.append([_actor, 
                      res_amount_dict[_actor], 
                      res_budge_dict[_actor], 
                      res_gross_dict[_actor]])
    
actors_data_sorted = sorted(actors_data, key=itemgetter(0), reverse=False)

hover_text = []

for entry in actors_data_sorted:
    hover_text.append(('<b>Actor: {actor}</b><br>'+
                       'Movies count: {count}<br>'+
                       'Budget: {budget}<br>'+
                       'Gross: {gross}<br>').format(
                                            actor=entry[0],
                                            count=entry[1],
                                            budget=hm.intword(entry[2]),
                                            gross=hm.intword(entry[3])
                        ))
    
amount = len(actors_data)
    
trace0 = go.Scatter(
    x = [budget for actor, budget in res_budge[:amount]],
    y = [gross for actor, gross in res_gross[:amount]],
    text = hover_text,
    mode='markers',
    marker=go.Marker(size = [amount for actor, amount in res_amount[:amount]],
                  sizemin = 2,
                  sizeref = 1.2,
                  sizemode ='diameter',
                  colorscale = 'Bluered', #https://github.com/plotly/plotly.js/blob/master/src/components/colorscale/scales.js
                  color = [amount for actor, amount in res_amount],
                  colorbar = dict(title = 'Amount of <br>movies'),)
)

layout = go.Layout(
    title = "Actors and their influence on business (mean)",
    hovermode="closest",
    
    xaxis=dict(tickangle=-45,
               title = "Budget USD"
              ),
    yaxis=dict(title = "Gross USD"),
    showlegend=False,
    height=700,
)

data = [trace0]
fig = go.Figure(data = data, layout=layout, )
iplot(fig)

write_graph(target_file, 
            plotly.offline.plot(fig, include_plotlyjs=False, output_type='div'),
           "actors/business mean")


In [38]:
def actors_and_gross_for_all_their_movies(movies_data):
    all_actors_gross = defaultdict(float)
    all_actors_budget = defaultdict(float)
    all_actors_amount = defaultdict(int)

    for title in movies_data:
        movie_info = movies_data[title]
        gross = float(movie_info["gross"])
        budget = float(movie_info["budget"])

        actor = movie_info["actor1"]
        all_actors_gross[actor] += gross
        all_actors_budget[actor] += budget
        all_actors_amount[actor] += 1

        actor = movie_info["actor2"]
        all_actors_gross[actor] += gross
        all_actors_budget[actor] += budget
        all_actors_amount[actor] += 1
        
        actor = movie_info["actor3"]
        all_actors_gross[actor] += gross
        all_actors_budget[actor] += budget
        all_actors_amount[actor] += 1
        
    return [
            all_actors_gross,
            all_actors_budget,
            all_actors_amount ]

result = actors_and_gross_for_all_their_movies(movies_main)

res_gross_dict = result[0]
res_budge_dict = result[1]
res_amount_dict = result[2]

res_gross = [(key, res_gross_dict[key]) for key in sorted(res_gross_dict.iterkeys())]
res_budge = [(key, res_budge_dict[key]) for key in sorted(res_budge_dict.iterkeys())]
res_amount = [(key, res_amount_dict[key]) for key in sorted(res_amount_dict.iterkeys())]

actors_data = []

for _actor in res_gross_dict:
    actors_data.append([_actor, 
                      res_amount_dict[_actor], 
                      res_budge_dict[_actor], 
                      res_gross_dict[_actor]])
    
actors_data_sorted = sorted(actors_data, key=itemgetter(0), reverse=False)

hover_text = []

for entry in actors_data_sorted:
    hover_text.append(('<b>Actor: {actor}</b><br>'+
                       'Movies count: {count}<br>'+
                       'Budget: {budget}<br>'+
                       'Gross: {gross}<br>').format(
                                            actor=entry[0],
                                            count=entry[1],
                                            budget=hm.intword(entry[2]),
                                            gross=hm.intword(entry[3])
                        ))
    
amount = len(actors_data)
    
trace0 = go.Scatter(
    x = [budget for actor, budget in res_budge[:amount]],
    y = [gross for actor, gross in res_gross[:amount]],
    text = hover_text,
    mode='markers',
    marker=go.Marker(size = [amount for actor, amount in res_amount[:amount]],
                  sizemin = 2,
                  sizeref = 1.2,
                  sizemode ='diameter',
                  colorscale = 'Bluered', #https://github.com/plotly/plotly.js/blob/master/src/components/colorscale/scales.js
                  color = [amount for actor, amount in res_amount],
                  colorbar = dict(title = 'Amount of <br>movies'),)
)

layout = go.Layout(
    title = "Actors and their influence on business",
    hovermode="closest",
    
    xaxis=dict(tickangle=-45,
               title = "Budget USD"
              ),
    yaxis=dict(title = "Gross USD"),
    showlegend=False,
    height=700,
)

data = [trace0]
fig = go.Figure(data = data, layout=layout, )
iplot(fig)

write_graph(target_file, 
            plotly.offline.plot(fig, include_plotlyjs=False, output_type='div'),
           "actors/business")


In [39]:
def actors_rank_gross(movies_data):
    all_actors_budget = defaultdict(float)
    all_actors_gross = defaultdict(float)
    
    for title in movies_data:
        movie_info = movies_data[title]
        gross = float(movie_info["gross"])
        budget = float(movie_info["budget"])
        
        actor = movie_info["actor1_rank"]
        all_actors_gross[actor] += gross
        all_actors_budget[actor] += budget
        
        actor = movie_info["actor2_rank"]
        all_actors_gross[actor] += gross
        all_actors_budget[actor] += budget
        
        actor = movie_info["actor3_rank"]
        all_actors_gross[actor] += gross
        all_actors_budget[actor] += budget
        
    return [
            all_actors_gross,
            all_actors_budget
            ]

result = actors_rank_gross(movies_main)

res_gross_dict = result[0]
res_budge_dict = result[1]

res_gross = [(key, res_gross_dict[key]) for key in sorted(res_gross_dict.iterkeys())]
res_budge = [(key, res_budge_dict[key]) for key in sorted(res_budge_dict.iterkeys())]

actors_data = []

for _actor in res_gross_dict:
    actors_data.append([_actor, 
                      res_budge_dict[_actor], 
                      res_gross_dict[_actor]])

actors_data_sorted = sorted(actors_data, key=itemgetter(0), reverse=False)

hover_text = []

for entry in actors_data_sorted:
    hover_text.append(('<b>Actor rank: {actor}</b><br>'+
                       'Budget: {budget}<br>'+
                       'Gross: {gross}<br>').format(
                                            actor=entry[0],
                                            budget=hm.intword(entry[1]),
                                            gross=hm.intword(entry[2])
                        ))
    
amount = len(actors_data)
    
trace0 = go.Scatter(
    x = [actor for actor, budget in res_budge[:amount]],
    y = [gross for actor, gross in res_gross[:amount]],
    text = hover_text,
    mode='markers',
    marker=go.Marker(
#                   size = [amount for actor, amount in res_amount[:amount]],
                  size = 3,
                  sizemode ='diameter',
                  colorscale = 'Bluered', #https://github.com/plotly/plotly.js/blob/master/src/components/colorscale/scales.js
))
    
layout = go.Layout(
    title = "Ranks vs gross (higher rank means more popular actor)",
    hovermode="closest",
    
    xaxis=dict(tickangle=-45,
               title = "Rank"
              ),
    yaxis=dict(title = "Gross USD"),
    showlegend=False,
    height=700,
)

data = [trace0]
fig = go.Figure(data = data, layout=layout, )
iplot(fig)

write_graph(target_file, 
            plotly.offline.plot(fig, include_plotlyjs=False, output_type='div'),
            "rank vs gross")

target_file.close()